In [1]:
import zmq
import numpy as np

# Remarks and caveats

A few things to be kept in mind (known bug-ish behavior :))
* The data sender will wait infinitely long to send data -- i.e, not good time out behavior implemented currently if the plottr isn't running.
* If the user closes the window with a given data ID, the plottr will not revive that window.

# Tools

In [82]:
port = 5557
addr = f"tcp://127.0.0.1:{port}" 

def send_data(data):
    context = zmq.Context()
    socket = context.socket(zmq.PUSH)
    socket.connect(addr)
    socket.send_json(data)
    socket.disconnect(addr)

# Send static data

We're sending a bunch of data in a few shapes to illustrate  plottr's basic behavior and UI. We see that we can change the data array being plotted, and we can choose against which axes to plot the data.

In [106]:
def make_1d_data(nx):
    x = np.linspace(0, 10, nx, dtype=float)
    y = np.cos(x)
    return x, y

def make_2d_data(nx, ny):
    x = np.linspace(0, 10, nx, dtype=float)
    y = np.linspace(-5, 5, ny, dtype=float)
    xx, yy = np.meshgrid(x, y, indexing='ij')
    zz = np.exp(-yy**2/(2 * 2**3)) * np.cos(xx)
    return xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)

def make_2d_avg_data(nx, ny, navg):
    x = np.array([])
    y = np.array([])
    z = np.array([])
    avg = np.array([])
    for i in range(navg):
        _x, _y, _z = make_2d_data(nx, ny)
        _z += (np.random.rand(nx, ny).reshape(-1) * 2. - 1.)
        x = np.append(x, _x)
        y = np.append(y, _y)
        z = np.append(z, _z)
        avg = np.append(avg, np.ones(_z.size) * i)
        
    return x, y, avg, z

In [123]:
# just a boring cos curve
x1, y1 = make_1d_data(21)

# a cos in x and a gauss curve in y
x2, y2, z2 = make_2d_data(31, 11)

# emulate a noise measurement of the cos/gauss data, and average a few times.
x3, y3, avg3, z3 = make_2d_avg_data(31, 11, 10)

# send all of that in one go.
data = {
    'id' : 'Dummy data no.1',
    'datasets' : {
        '1d sine' : {
            'values' : y1.tolist(),
            'axes' : ['x-axis 1',],
        },
        'x-axis 1' : {
            'values' : x1.tolist(),
        },
        
        
        'cos and gauss' : {
            'values' : z2.tolist(),
            'axes' : ['osc', 'gauss', ],
        },
        'osc' : {
            'values' : x2.tolist(),
        },
        'gauss' : {
            'values' : y2.tolist(),
        },
        
        
        'cos, gauss, avg' : {
            'values' : z3.tolist(),
            'axes' : ['x vals', 'y vals', 'averages'],
        },
        'x vals' : {
            'values' : x3.tolist(),
        },
        'y vals' : {
            'values' : y3.tolist(),
        },
        'averages' : {
            'values' : avg3.tolist(),
        },
        
    }
}

send_data(data)

Sending another dataset (i.e., something with a different data ID) will trigger a new data window.

In [122]:
data2 = data.copy()
data2.update(dict(id='Dummy data no.1 copy'))
send_data(data2)